In [10]:
import requests
# For saving access tokens and for file management when creating and adding to the dataset
import os
# For dealing with json responses we receive from the API
import json
# For displaying the data after
import pandas as pd
# For saving the response data in CSV format
import csv
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
#To add wait time between requests
import time

In [11]:
os.environ['TOKEN'] = 'AAAAAAAAAAAAAAAAAAAAAPfEhwEAAAAAElz971Kl5tLPtcriYImZyMc7TbA%3D7BSUXbf9ZESoLGtCug0zWwoT0dsN7gNfc5f9fTKTjzSA8v92BR'

In [12]:
def auth():
    return os.getenv('TOKEN')

In [13]:
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

In [14]:
def create_url(start_date, max_results, tweet_fields, expansions, end_time=None):
    
    search_url = "https://api.twitter.com/2/users/1162694149956603904/mentions" #Change to the endpoint you want to collect data from

    #change params based on the endpoint you are using
    query_params = {'tweet.fields' : tweet_fields,
                    'start_time': start_date,
                    'end_time' : end_time,
                    'max_results': max_results,
                    'expansions': expansions,
                    'next_token': {}}
    return (search_url, query_params)

In [15]:
def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [16]:
start_date = '2022-06-12T00:00:00.000Z'
# end_time = "2022-06-29T00:00:00.000Z"
max_results = 100
tweet_fields = 'created_at,public_metrics'
expansions = 'author_id'
bearer_token = auth()
headers = create_headers(bearer_token)

In [17]:
url = create_url(start_date, max_results, tweet_fields, expansions)

In [18]:
json_response = connect_to_endpoint(url[0], headers, url[1])

Endpoint Response Code: 200


In [27]:
retweet_count = json_response['data'][3]['public_metrics']['retweet_count']
reply_count = json_response['data'][3]['public_metrics']['reply_count']
like_count = json_response['data'][3]['public_metrics']['like_count']
quote_count = json_response['data'][3]['public_metrics']['quote_count']

In [30]:
json_response['meta']['result_count']

100

In [9]:
def save_json(number, json_response):
    filename = 'data' + str(number) + '.json'
    path = 'data/' + filename
    with open(path, 'w') as f:
        json.dump(json_response, f)

In [331]:
def get_all_tweets(start_date, max_results, tweet_fields, expansions, end_time=None):
    url = create_url(start_date, max_results, tweet_fields, expansions)
    json_response = connect_to_endpoint(url[0], headers, url[1])
    save_json(1, json_response)
    end_time = json_response['data'][-1]['created_at']
    url = create_url(start_date, max_results, tweet_fields, expansions, end_time)
    json_response = connect_to_endpoint(url[0], headers, url[1])
    save_json(2, json_response)
    return json_response

In [332]:
json_response = get_all_tweets(start_date, max_results, tweet_fields, expansions)

Endpoint Response Code: 200
2022-06-28T07:50:34.000Z


In [333]:
# print(json.dumps(json_response, indent=4, sort_keys=True))

In [1]:
import sqlite3

In [10]:
connection = sqlite3.connect("../data/twitter.db")
crsr = connection.cursor()

In [13]:
def sql_query(query):

    # Ejecuta la query
    crsr.execute(query)

    # Almacena los datos de la query 
    ans = crsr.fetchall()

    # Obtenemos los nombres de las columnas de la tabla
    names = [description[0] for description in crsr.description]

    return pd.DataFrame(ans,columns=names)

In [16]:
query_match = '''SELECT * FROM users'''

In [17]:
sql_query(query_match)

,id,name,username
0,1578095844569514011,Rocket,75_Rocket_
1,1003872445,Pau Mugarra,PauMugarra
2,2529499620,Formación Fundación Universidad Carlos III,formacion_fuc3
3,706520411551494145,JoseSola,Jose_Sola_
4,604485175,Berta Otero,bertinha84
5,1391838162830385165,Apiux Tecnología,Apiuxtecnologia
6,1494355208473874438,PyData Madrid,PyDataMadrid
7,12085582,diegodl,diegodl
8,2907682397,graphext,graphext
9,1131933723681800193,Biohub VLC,BiohubVLC
